In [ ]:
import os
from dotenv import load_dotenv
from pprint import pprint

from langchain.globals import set_verbose, set_debug
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser

from utils import process_and_store_documents, build_graph
from config import LOCAL_LLM, URLS, RECURSION_LIMIT
from config import RetrievalGrader_prompt, generate_prompt, HallucinationGrader_prompt, AnswerGrader_prompt, Router_prompt
from nodes import Nodes

load_dotenv()

set_debug(True)
set_verbose(True)

In [ ]:
retriever = process_and_store_documents(URLS)

In [ ]:
retrieval_grader = RetrievalGrader_prompt | LOCAL_LLM | JsonOutputParser()
question = "agent memory"
docs = retriever.invoke(question)
doc_txt = docs[1].page_content
print(retrieval_grader.invoke({"question": question, "document": doc_txt}))

In [ ]:
# Post-processing
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

# Chain
rag_chain = generate_prompt | LOCAL_LLM | StrOutputParser()

# Run
question = "agent memory"
docs = retriever.invoke(question)
generation = rag_chain.invoke({"context": docs, "question": question})
print(generation)

In [ ]:
hallucination_grader = HallucinationGrader_prompt | LOCAL_LLM | JsonOutputParser()
hallucination_grader.invoke({"documents": docs, "generation": generation})

In [ ]:
answer_grader = AnswerGrader_prompt | LOCAL_LLM | JsonOutputParser()
answer_grader.invoke({"question": question,"generation": generation})

In [ ]:
question_router = Router_prompt | LOCAL_LLM | JsonOutputParser()
question = "llm agent memory"
docs = retriever.get_relevant_documents(question)
doc_txt = docs[1].page_content
print(question_router.invoke({"question": question}))

In [8]:
### Search
os.environ["TAVILY_API_KEY"] = "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"

from langchain_community.tools.tavily_search import TavilySearchResults
web_search_tool = TavilySearchResults(k=3)

In [9]:
nodes = Nodes(
        retriever=retriever,
        rag_chain=rag_chain,
        retrieval_grader=retrieval_grader,
        web_search_tool=web_search_tool,
        question_router=question_router,
        hallucination_grader=hallucination_grader,
        answer_grader=answer_grader
    )

workflow = build_graph(nodes)
app = workflow.compile()    # Compile

In [ ]:
from IPython.display import Image, display

try:
    display(Image(app.get_graph(xray=True).draw_mermaid_png()))
except Exception:
    print("Display error!")
    pass

In [ ]:
# Test

inputs = {"question": "What are the types of agent memory?"}


for output in app.stream(inputs, RECURSION_LIMIT):
    for key, value in output.items():
        pprint(f"Finished running: {key}:")
pprint(value["generation"])